# GCC's ssc-cdi : ptychography pipeline for Cateretê v0.1
 
- #### Questions? E-mail gcc@lnls.br
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/wiki/

## Imports

In [3]:
%matplotlib widget
import os, json, h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sscInstall
import sscCdi
import sscResolution

print(f'sscCdi version: {sscCdi.__version__}')
print(f'sscResolution version: {sscResolution.__version__}')

sscCdi version: 0.6.20
sscResolution version: 0.5.8


## Inputs

In [6]:
input_dict = {
    
'beamline': 'CAT',
'detector': '540D',
'debug': False, # use False

"data_folder": "/ibira/lnls/beamlines/caterete/proposals/20222150/data/ptycho3d/",
"acquisition_folders": ["A1_01", "A1_03"],
"projections": [195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418],

"flatfield": "",

'CPUs': 96,
'GPUs': [0,1,2],

'binning': 1, 
'using_direct_beam': False, # if True, converts DP_center coordinates from raw to restored coordinates
'DP_center': [1392,1405],  # [center_y, center_x]
'fill_blanks': False, # interpolate blank lines after restoration
'keep_original_negative_values': False, 
'suspect_border_pixels': 3, # pixels
'detector_ROI_radius': 1350, # pixels

'position_rotation': -0.003, # radians. angle between detector and probe coordinate system

'object_padding': 50, # pixels

'incoherent_modes': 0, 
'probe_support': [ "circular", 300,0,0 ], # ["circular",radius_pxls,center_y, center_x]; (0,0) is the center of the image
'fresnel_number': -0.001,

"initial_obj": ["random"], # path to .npy, .hdf5, ["random"], ["constant"]
"initial_probe": ["inverse"], # path to .npy, .hdf5, ["inverse"], ["random"], ["constant"], ["circular",radius]

'Algorithm1': {'Batch': 64,
                'Beta': 0.995,
                'Epsilon': 0.01,
                'Iterations': 70,
                'Name': 'RAAR',
                'ProbeCycles': 4,
                'TV': 0},

'Algorithm2': {'Batch': 64,
                'Epsilon': 0.01,
                'Iterations': 50,
                'Name': 'GL',
                'ObjBeta': 0.97,
                'ProbeBeta': 0.95,
                'TV': 0.0001},

'Algorithm3': {'Batch': 64,
                'Epsilon': 0.1,
                'Iterations': 5,
                'Name': 'GL',
                'ObjBeta': 0.5,
                'ProbeBeta': 0.5,
                'TV': 0.01},
}

input_dict = sscCdi.caterete.cat_ptycho_processing.define_paths(input_dict)

	Proposal path:  /ibira/lnls/beamlines/caterete/proposals/20222150/data/ptycho3d/
	Acquisition folder:  A1_01
	Output path: /ibira/lnls/beamlines/caterete/proposals/20222150/proc/recons/A1_01


In [13]:
from sscCdi import list_files_in_folder, select_specific_angles

for acquisitions_folder in input_dict['acquisition_folders']:
    filepaths0, filenames0 = list_files_in_folder(os.path.join(input_dict['data_folder'], acquisitions_folder,input_dict['scans_string']), look_for_extension=".hdf5")
    
    print(filenames0)
    print(input_dict['projections'])
    
    # filepaths, filenames = select_specific_angles(input_dict['projections'], filepaths0,  filenames0)
    # print(filenames)

['0000_A1_01_001.hdf5', '0001_A1_01_001.hdf5', '0002_A1_01_001.hdf5', '0003_A1_01_001.hdf5', '0004_A1_01_001.hdf5', '0005_A1_01_001.hdf5', '0006_A1_01_001.hdf5', '0007_A1_01_001.hdf5', '0008_A1_01_001.hdf5', '0009_A1_01_001.hdf5', '0010_A1_01_001.hdf5', '0011_A1_01_001.hdf5', '0012_A1_01_001.hdf5', '0013_A1_01_001.hdf5', '0014_A1_01_001.hdf5', '0015_A1_01_001.hdf5', '0016_A1_01_001.hdf5', '0017_A1_01_001.hdf5', '0018_A1_01_001.hdf5', '0019_A1_01_001.hdf5', '0020_A1_01_001.hdf5', '0021_A1_01_001.hdf5', '0022_A1_01_001.hdf5', '0023_A1_01_001.hdf5', '0024_A1_01_001.hdf5', '0025_A1_01_001.hdf5', '0026_A1_01_001.hdf5', '0027_A1_01_001.hdf5', '0028_A1_01_001.hdf5', '0029_A1_01_001.hdf5', '0030_A1_01_001.hdf5', '0031_A1_01_001.hdf5', '0032_A1_01_001.hdf5', '0033_A1_01_001.hdf5', '0034_A1_01_001.hdf5', '0035_A1_01_001.hdf5', '0036_A1_01_001.hdf5', '0037_A1_01_001.hdf5', '0038_A1_01_001.hdf5', '0039_A1_01_001.hdf5', '0040_A1_01_001.hdf5', '0041_A1_01_001.hdf5', '0042_A1_01_001.hdf5', '0043_A1_0

## Load diffraction data to find center

In [3]:
# path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/scans/0000_SS03112022_02_001.hdf5" # enter path to raw diffractiom dataset

# fig, ax = plt.subplots(dpi=100)
# ax.imshow(np.squeeze(h5py.File(path,'r')['entry/data/data'])[0],norm=LogNorm(),cmap='jet')

# Run new job at cluster

In [4]:
# server = sscInstall.connect_server()    

In [5]:
json_filepath_path = sscCdi.cat_ptycho_processing.save_input_dictionary(input_dict)
# sscCdi.run_at_cluster(server,json_filepath_path,queue='cat',gpus=input_dict["GPUs"],cpus=input_dict["CPUs"],jobName='JOB',script_path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/ssc-cdi-0.6.17/bin/caterete_ptycho.py",slurm_path = '/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/inputs/')

# Run Locally

## Restoration

In [6]:
restoration_dict_list, restored_data_info_list = sscCdi.caterete.cat_restoration.restoration_ptycho_CAT(input_dict) # restoration of all frames; restored DPs saved at temporary folder

	Restoration of folder SS03112022_02

	Using 1 of 1 angle(s)


## Ptychography

In [7]:
input_dict, obj, probe, probe_positions = sscCdi.caterete.cat_ptycho_processing.cat_ptychography(input_dict,restoration_dict_list,restored_data_info_list)
obj, probe, angles, positions, error = sscCdi.misc.save_volume_from_parts(input_dict)
sscCdi.misc.save_variable(input_dict, obj,name='object')
sscCdi.misc.save_variable(input_dict,probe,name='probe')
sscCdi.misc.save_variable(input_dict,positions,name='positions')
sscCdi.misc.save_variable(input_dict,angles,name='angles')
sscCdi.misc.save_variable(input_dict,error,name='error',group='log')
sscCdi.misc.save_json_logfile(input_dict) 
sscCdi.misc.delete_temporary_folders(input_dict)



Reading diffraction data for angle: 0

Binning data...

	Finished reading diffraction data! DPs shape: (625, 674, 674)

Reading probe positions...

	Object pixel size = 19.51 nm

	Limit thickness for resolution of 1 pixel: 12.079 microns

	Finished reading probe positions. Shape: (625, 2)

	Initial object shape: (1, 2006, 2006)	 Initial probe shape: (1, 1, 674, 674)

	Estimated size for 1 DPs of type float32: 1.14 GBs

Fresnel number: -0.001

Creating initial probe...

	Probe shape (674,674) with 1 incoherent mode(s)

Setting probe support...

Creating initial object...

Diffraction Patterns: (625, 674, 674)

Initial Object: (2006, 2006)

Initial Probe: (1, 674, 674)

Probe Support: (1, 674, 674)

Probe Positions: (625, 4)

Creating datapack...

Total datapack size: 1.18 GBs

Starting ptychography... using 4 GPUs [0, 1, 2, 3] and 128 CPUs

	Calling 70 iterations of RAAR algorithm...

	Done in: 4.76 seconds

	Calling 50 iterations of GL algorithm...

	Done in: 1.88 seconds

	Calling 5

In [8]:
if 1: # phase
    preview = np.angle(obj)
else: # magnitude
    preview = np.abs(obj)

# sscCdi.misc.visualize_magnitude_and_phase(obj)
sscCdi.misc.deploy_visualizer(preview,title='Sinogram',cmap='gray')

## Crop object

In [ ]:
input_dict['crop'] = [1400,2000,1300,1900]
cropped_object = sscCdi.caterete.cat_ptycho_processing.crop_sinogram(input_dict,obj,probe_positions)
if 1: # phase
    cropped_object = np.angle(cropped_object)
else: # magnitude
    cropped_object = np.abs(cropped_object)

In [ ]:
sscCdi.misc.deploy_visualizer(cropped_object,title='Cropped sinogram',cmap='gray')

## Unwrap object

In [ ]:
phase = sscCdi.caterete.unwrap_in_parallel(cropped_object)
sscCdi.misc.save_variable(input_dict,phase, name = 'object_unwrapped')

In [ ]:
sscCdi.misc.deploy_visualizer(phase,title='Unwrapped',cmap='gray')

## Fourier Ring Correlation

In [ ]:
frame = 0

if 1: # FRC on phase image
    img = phase[frame]
else: # else, on absorption image
    img = np.abs(cropped_object)[frame]

padding = 0
sharpness = 10
radius = 0.8

dic = sscResolution.frc.computep( sscResolution.frc.window( img, padding, [sharpness, radius] ) , input_dict["CPUs"] ) 
print(f"\tResolution via halfbit criterion: {1e9*input_dict['object_pixel']/dic['x']['H']:.2f} nm")

sscResolution.frc.plot(dic, {'label': "Resolution", 'unit': "nm", 'pxlsize': input_dict['object_pixel']*1e9} )